In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
from plotly.graph_objs import Layout, XAxis, YAxis

import orchest
import pickle
import json
import os

In [ ]:
orchest_data = orchest.get_inputs()
response = orchest_data["response"]

In [ ]:
orchest_data = orchest.get_inputs()
response = orchest_data["response"]
try:
    dates = response.index
    values = sum(response.values.tolist(), [])
    print("Data received: ")
except Exception as e:
    print(e)

In [ ]:
df = pd.DataFrame(data = {'date': dates, 'x': values})
df.head()

In [ ]:
fig = go.Figure(data=go.Scatter(x = df['date'], y = df['x'], mode='markers', marker=dict(color='blue', size=5)))
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [ ]:
scaler = StandardScaler()
np_scaled = scaler.fit_transform(df[["x"]])
data = pd.DataFrame(np_scaled)

In [ ]:
print("Load the model from disk...")
file = '../data/model.sav'
model = pickle.load(open(file, 'rb'))

In [ ]:
print("Do anomaly detection...")
df['anomaly'] = pd.Series(model.predict(data))
anomaly = df.loc[df['anomaly'] == -1, ['date', 'x']] 
print(model.get_params())

In [ ]:
layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)', 
    xaxis=XAxis(showgrid=True, zeroline=True, showline=True, zerolinecolor='#DBDBDB', zerolinewidth=2, gridcolor='#DBDBDB', gridwidth=2,  linecolor='#AFAFAF', linewidth=2),
    yaxis=YAxis(showgrid=True, zeroline=True, showline=True, zerolinecolor='#DBDBDB', zerolinewidth=2, gridcolor='#DBDBDB', gridwidth=2, linecolor='#AFAFAF', linewidth=2)
)

fig = go.Figure(data=go.Scatter(x = df['date'], y = df['x'], mode='lines', name = "Normal values", marker=dict(color='blue', size=2)), layout = layout)
fig.add_traces(go.Scatter(x = anomaly['date'], y = anomaly['x'], textposition='top left', mode='markers+text', name = "Anomaly value", marker=dict(color='red', size=6)))

fig.show()

In [ ]:
if not anomaly.empty:
    print("Anomaly points exist")
    fig.write_image("../data/plot.pdf")
else:
    print("No anomaly points")

In [ ]:
orchest.output((anomaly), name = "anomaly")
print("Done!")